# MNIST Large Untrained Net Exc Inh

Derived from https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb

In [135]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from keras import backend as K
#print("keras version:",tf.keras.__version__)

TensorFlow version: 2.8.2


In [136]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

In [137]:
positiveWeightImplementation = False	#orig: True

generateUntrainedNetwork = False
if(generateUntrainedNetwork):
    #only train the last layer
    numberOfHiddenLayers = 2    #default: 2    #if 0 then useSVM=True
else:
    numberOfHiddenLayers = 2   #default: 2

if(numberOfHiddenLayers > 1):
    addSkipLayers = False   #optional
else:
    addSkipLayers = False   #mandatory

layerSizeBase = 128  #default: 128

batch_size = 64 #default: 64
epochs = 5  #1  #5

debugPreTrainWeights = False
debugPreTrainOutputs = False
debugPostTrainWeights = False
debugPostTrainOutputs = False

## Load data

In [138]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print("x_train.shape = ", x_train.shape)

input_shape = (28, 28)

x_train.shape =  (60000, 28, 28)


## Define model

In [139]:
def activationExcitatory(x):
    return K.maximum(x, 0)  #ReLU

def activationInhibitory(x):
    if(positiveWeightImplementation):
        return -(K.maximum(x, 0))   #ReLU with negative output
    else:
        return K.maximum(x, 0)  #ReLU

def excitatoryNeuronInitializer(shape, dtype=None):
    return tf.math.abs(tf.random.normal(shape, dtype=dtype))

def inhibitoryNeuronInitializer(shape, dtype=None):
    if(positiveWeightImplementation):
        return tf.math.abs(tf.random.normal(shape, dtype=dtype))
    else:
        return tf.math.negative(tf.math.abs(tf.random.normal(shape, dtype=dtype)))

class negative(tf.keras.constraints.Constraint):
    #based on https://www.tensorflow.org/api_docs/python/tf/keras/constraints/Constraint
    def __init__(self):
        pass
    def __call__(self, w):
        return w * tf.cast(tf.math.less_equal(w, 0.), w.dtype)


In [140]:
num_classes = 10

if(positiveWeightImplementation):
    weightConstraint = tf.keras.constraints.non_neg()
    constrainBiases = True   #ensure positive biases also
    if(constrainBiases):
        biasConstraint = tf.keras.constraints.non_neg()
        constrainBiasesLastLayer = False
        if(constrainBiasesLastLayer):
            biasConstraintLastLayer = tf.keras.constraints.non_neg()
        else:
            biasConstraintLastLayer = None
    else:
        biasConstraint = None
        biasConstraintLastLayer = None
    weightConstraintLastLayer = weightConstraint
else:
    weightConstraintPositive = tf.keras.constraints.non_neg()
    weightConstraintNegative = negative()
    constrainBiases = False
    if(constrainBiases):
        biasConstraintPositive = tf.keras.constraints.non_neg()
        biasConstraintNegative = negative()
    else:
        biasConstraintPositive = None
        biasConstraintNegative = None
    weightConstraintLastLayer = None
    biasConstraintLastLayer = None

if(generateUntrainedNetwork):
    #only train the last layer
    generateLargeNetwork = True
else:
    generateLargeNetwork = False


if(generateLargeNetwork):
    generateLargeNetworkRatio = 50
    layerRatio = generateLargeNetworkRatio
else:
    layerRatio = 1  #10 #1

def createEIlayer(layerIndex, h0, firstLayer=False):
    if(positiveWeightImplementation):
        h1E = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=excitatoryNeuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h0)
        h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=inhibitoryNeuronInitializer, kernel_constraint=weightConstraint, bias_constraint=biasConstraint)(h0)
        h1E = tf.keras.layers.Activation(activationExcitatory)(h1E)
        h1I = tf.keras.layers.Activation(activationInhibitory)(h1I)
        h1 = tf.keras.layers.Concatenate()([h1E, h1I])
    else:
        if(firstLayer):
            h1E = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=excitatoryNeuronInitializer)(h0)   #excitatory neuron inputs
            h1I = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=inhibitoryNeuronInitializer)(h0)   #inhibitory neuron inputs
        else:
            h0E, h0I = h0
            h1Ee = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=excitatoryNeuronInitializer, kernel_constraint=weightConstraintPositive, bias_constraint=biasConstraintPositive)(h0E) #excitatory neuron excitatory inputs
            h1Ei = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=excitatoryNeuronInitializer, kernel_constraint=weightConstraintNegative, bias_constraint=biasConstraintNegative)(h0I) #excitatory neuron inhibitory inputs
            h1Ie = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=inhibitoryNeuronInitializer, kernel_constraint=weightConstraintPositive, bias_constraint=biasConstraintPositive)(h0E) #inhibitory neuron excitatory inputs
            h1Ii = tf.keras.layers.Dense(layerSizeBase*layerRatio, kernel_initializer=inhibitoryNeuronInitializer, kernel_constraint=weightConstraintNegative, bias_constraint=biasConstraintNegative)(h0I) #inhibitory neuron inhibitory inputs
            h1E = tf.keras.layers.Add()([h1Ee, h1Ei])
            h1I = tf.keras.layers.Add()([h1Ie, h1Ii])
        h1E = tf.keras.layers.Activation(activationExcitatory)(h1E)
        h1I = tf.keras.layers.Activation(activationInhibitory)(h1I)
        h1 = (h1E, h1I)
    return h1

def concatEIneurons(h):
    if(positiveWeightImplementation):
        return h
    else: 
        hE, hI = h
        h = tf.keras.layers.Concatenate()([hE, hI])
        return h

x = tf.keras.layers.Input(shape=input_shape)
h0 = tf.keras.layers.Flatten()(x)

hLast = h0
if(numberOfHiddenLayers >= 1):
    h1 = createEIlayer(1, h0, firstLayer=True)
    hLast = h1
if(numberOfHiddenLayers >= 2):
    h2 = createEIlayer(2, h1)
    hLast = h2
if(numberOfHiddenLayers >= 3):
    h3 = createEIlayer(3, h2)
    hLast = h3
if(numberOfHiddenLayers >= 4):
    h4 = createEIlayer(4, h3)
    hLast = h4
if(addSkipLayers):
    mList = []
    if(numberOfHiddenLayers >= 1):
        m1 = tf.keras.layers.Flatten()(concatEIneurons(h1))
        mList.append(m1)
    if(numberOfHiddenLayers >= 2):
        m2 = tf.keras.layers.Flatten()(concatEIneurons(h2))
        mList.append(m2)
    if(numberOfHiddenLayers >= 3):
        m3 = tf.keras.layers.Flatten()(concatEIneurons(h3))
        mList.append(m3)
    if(numberOfHiddenLayers >= 4):
        m4 = tf.keras.layers.Flatten()(concatEIneurons(h4))
        mList.append(m4)
    hLast = tf.keras.layers.concatenate(mList)
else:
    hLast = concatEIneurons(hLast)

if(generateUntrainedNetwork):
    hLast = tf.keras.layers.Lambda(lambda x: tf.keras.backend.stop_gradient(x))(hLast)
y = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_constraint=weightConstraintLastLayer, bias_constraint=biasConstraintLastLayer)(hLast)
model = tf.keras.Model(x, y)

#print(model.summary())
#model.compile(optimizer=tf.keras.optimizers.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])
#evaluation accuracy: ? (with 1 or 2 hidden layers)


In [141]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [142]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

print(model.summary())

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 28, 28)]     0           []                               
                                                                                                  
 flatten_9 (Flatten)            (None, 784)          0           ['input_10[0][0]']               
                                                                                                  
 dense_62 (Dense)               (None, 128)          100480      ['flatten_9[0][0]']              
                                                                                                  
 dense_63 (Dense)               (None, 128)          100480      ['flatten_9[0][0]']              
                                                                                            

In [143]:
if(debugPreTrainWeights):
    testwritefile = open('weights.txt', 'a')
    for layerIndex, layer in enumerate(model.layers):
        heading = "layerWeights = " + str(layerIndex) + "\n"
        testwritefile.write(heading)
        #print(heading)
        weights = layer.get_weights()
        #print(weights)
        weightsS =  str(weights)
        testwritefile.write(weightsS)

In [144]:
if(debugPreTrainOutputs):
    #testwritefile = open('output.txt', 'a')
    xTrainFirstSample = np.expand_dims(x_train[0], axis=0)
    for layerIndex, layer in enumerate(model.layers):
        heading = "layerOutputs = " + str(layerIndex) + "\n"
        print(heading)
        #testwritefile.write(heading)
        func = K.function([model.get_layer(index=0).input], layer.output)
        layerOutput = func([xTrainFirstSample])  # input_data is a numpy array
        print(layerOutput)
        #layerOutputS =  str(layerOutput)  #tf.tensor.toString(layerOutput)    #layerOutput.tostring()
        #testwritefile.write(layerOutputS)

## Train model

In [145]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/5
938/938 [==============================] - 3s 3ms/step - loss: 261.4042 - accuracy: 0.3773
Epoch 2/5
938/938 [==============================] - 3s 3ms/step - loss: 47.1078 - accuracy: 0.6230
Epoch 3/5
938/938 [==============================] - 3s 3ms/step - loss: 36.7385 - accuracy: 0.7003
Epoch 4/5
938/938 [==============================] - 3s 3ms/step - loss: 35.5374 - accuracy: 0.7351
Epoch 5/5
938/938 [==============================] - 3s 3ms/step - loss: 33.3302 - accuracy: 0.7684


In [146]:
if(debugPostTrainWeights):
    testwritefile = open('weights.txt', 'a')
    for layerIndex, layer in enumerate(model.layers):
        heading = "layerWeights = " + str(layerIndex) + "\n"
        testwritefile.write(heading)
        #print(heading)
        weights = layer.get_weights()
        #print(weights)
        weightsS =  str(weights)
        testwritefile.write(weightsS)

In [147]:
if(debugPostTrainOutputs):
    #testwritefile = open('output.txt', 'a')
    xTrainFirstSample = np.expand_dims(x_train[0], axis=0)
    for layerIndex, layer in enumerate(model.layers):
        heading = "layerOutputs = " + str(layerIndex) + "\n"
        print(heading)
        #testwritefile.write(heading)
        func = K.function([model.get_layer(index=0).input], layer.output)
        layerOutput = func([xTrainFirstSample])  # input_data is a numpy array
        print(layerOutput)
        #layerOutputS =  str(layerOutput)  #tf.tensor.toString(layerOutput)    #layerOutput.tostring()
        #testwritefile.write(layerOutputS)

## Evaluate model

In [148]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 38.8289 - accuracy: 0.7483 - 863ms/epoch - 3ms/step


[38.828895568847656, 0.7483000159263611]

In [149]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [150]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.08533675, 0.08533675, 0.08533675, 0.08533675, 0.08533675,
        0.08533675, 0.08533675, 0.23196931, 0.08533675, 0.08533675],
       [0.08533675, 0.08533675, 0.23196931, 0.08533675, 0.08533675,
        0.08533675, 0.08533675, 0.08533675, 0.08533675, 0.08533675],
       [0.08533675, 0.23196931, 0.08533675, 0.08533675, 0.08533675,
        0.08533675, 0.08533675, 0.08533675, 0.08533675, 0.08533675],
       [0.23196931, 0.08533675, 0.08533675, 0.08533675, 0.08533675,
        0.08533675, 0.08533675, 0.08533675, 0.08533675, 0.08533675],
       [0.08533675, 0.08533675, 0.08533675, 0.08533675, 0.23196931,
        0.08533675, 0.08533675, 0.08533675, 0.08533675, 0.08533675]],
      dtype=float32)>